In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./menu.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                if len(browser.find_elements(by=By.XPATH, value='//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]')) != 0:
                    break
                
                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//tbody[contains(@class, "listing-inner") and contains(@class, "altrow-a-1")]')))
                list_part = browser.find_elements(by=By.XPATH, value='//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]') for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                part_index = [_.text for _ in list_part_number].index(url.split(',')[-3])
                
                # = = = = = = = = = = = = = = =
                
                manufacturer = list_part[part_index].find_element(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]').text
                
                # = = = = = = = = = = = = = = =
                
                list_part_number[part_index].click()
                time.sleep(7)
                list_vehicle = browser.find_element(by=By.XPATH, value='//div[contains(@id, "buyersguidepopup-outer_c")]').find_elements(by=By.XPATH, value='./descendant::table[contains(@class, "nobmp")]')
                if len(list_vehicle) == 0:
                    vehicle = ''
                else:
                    vehicle = list_vehicle[0].text
                
                # = = = = = = = = = = = = = = =
                
                note_1 = ';'.join(_.text for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]'))
                
                # = = = = = = = = = = = = = = =
                
                info = ';'.join(_.get_attribute('href') for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]'))
                
                # = = = = = = = = = = = = = = =
                
                note_2 = ';'.join(_.text for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]'))
                
                # = = = = = = = = = = = = = = =
                
                oe = ';'.join(_.get_attribute('textContent') for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]'))
                oe = ';'.join(oe.split(', '))
                
                # = = = = = = = = = = = = = = =
                
                src = ';'.join(_.get_attribute('src') for _ in list_part[part_index].find_elements(by=By.XPATH, value='./descendant::img[contains(@id, "inlineimg_thumb")]'))
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_number': url.split(',')[-3],
                                         'Manufacturer': manufacturer,
                                         'Vehicle': vehicle,
                                         'Alternate_OE_Part_Numbers': oe,
                                         'Note_1': note_1,
                                         'Note_2': note_2,
                                         'Pic': '',
                                         'Url': url,
                                         'Info': info,
                                         'Src': src}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_number': url.split(',')[-3],
                                     'Url': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url.split(',')[-3] +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(7):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./part.xlsx', index=False)
print()
print('爬虫结束')

总数量：842

3047  <->  [ok] - 剩余数量：835
3037  <->  [ok] - 剩余数量：834
3032  <->  [ok] - 剩余数量：833
3072  <->  [ok] - 剩余数量：832
3002  <->  [ok] - 剩余数量：831
3046  <->  [ok] - 剩余数量：830
3009  <->  [ok] - 剩余数量：829
3079  <->  [ok] - 剩余数量：828
3080  <->  [ok] - 剩余数量：827
3108  <->  [ok] - 剩余数量：826
3081  <->  [ok] - 剩余数量：825
3118  <->  [ok] - 剩余数量：824
30001  <->  [ok] - 剩余数量：823
30002  <->  [ok] - 剩余数量：822
30003C  <->  [ok] - 剩余数量：821
30007C  <->  [ok] - 剩余数量：820
30006C  <->  [ok] - 剩余数量：819
30005C  <->  [ok] - 剩余数量：818
30004  <->  [ok] - 剩余数量：817
30008C  <->  [ok] - 剩余数量：816
30009C  <->  [ok] - 剩余数量：815
30014  <->  [ok] - 剩余数量：814
3000C  <->  [ok] - 剩余数量：813
30015C  <->  [ok] - 剩余数量：812
30017C  <->  [ok] - 剩余数量：811
30016C  <->  [ok] - 剩余数量：810
30019C  <->  [ok] - 剩余数量：809
30013  <->  [ok] - 剩余数量：808
30020C  <->  [ok] - 剩余数量：807
3001C  <->  [ok] - 剩余数量：806
30021C  <->  [ok] - 剩余数量：805
30024  <->  [ok] - 剩余数量：804
30022  <->  [ok] - 剩余数量：803
30024C  <->  [ok] - 剩余数量：802
30025  <->  [ok] - 剩余数量：801
30026C  <-

3385C  <->  [ok] - 剩余数量：545
3386C  <->  [ok] - 剩余数量：544
3390C  <->  [ok] - 剩余数量：543
3391C  <->  [ok] - 剩余数量：542
3392C  <->  [ok] - 剩余数量：541
3388C  <->  [ok] - 剩余数量：540
3394  <->  [ok] - 剩余数量：539
3393C  <->  [ok] - 剩余数量：538
3396  <->  [ok] - 剩余数量：537
3397C  <->  [ok] - 剩余数量：536
3396C  <->  [ok] - 剩余数量：535
3395C  <->  [ok] - 剩余数量：534
3398C  <->  [ok] - 剩余数量：533
3399C  <->  [ok] - 剩余数量：532
3405  <->  [ok] - 剩余数量：531
3420C  <->  [ok] - 剩余数量：530
3423  <->  [ok] - 剩余数量：529
3424C  <->  [ok] - 剩余数量：528
3422C  <->  [ok] - 剩余数量：527
3431  <->  [ok] - 剩余数量：526
3443C  <->  [ok] - 剩余数量：525
3444  <->  [ok] - 剩余数量：524
3438C  <->  [ok] - 剩余数量：523
3457C  <->  [ok] - 剩余数量：522
3462C  <->  [ok] - 剩余数量：521
3445C  <->  [ok] - 剩余数量：520
3467C  <->  [ok] - 剩余数量：519
3468C  <->  [ok] - 剩余数量：518
3474C  <->  [ok] - 剩余数量：517
3478C  <->  [ok] - 剩余数量：516
3490C  <->  [ok] - 剩余数量：515
3480  <->  [ok] - 剩余数量：514
3481C  <->  [ok] - 剩余数量：513
3491C  <->  [ok] - 剩余数量：512
3500C  <->  [ok] - 剩余数量：511
3493C  <->  [ok] - 剩余数量：510

4277C  <->  [ok] - 剩余数量：250
4279  <->  [ok] - 剩余数量：249
4282C  <->  [ok] - 剩余数量：248
4283C  <->  [ok] - 剩余数量：247
4290C  <->  [ok] - 剩余数量：246
4284C  <->  [ok] - 剩余数量：245
4291C  <->  [ok] - 剩余数量：244
4293C  <->  [ok] - 剩余数量：243
4295C  <->  [ok] - 剩余数量：242
4298C  <->  [ok] - 剩余数量：241
4299C  <->  [ok] - 剩余数量：240
4297C  <->  [ok] - 剩余数量：239
4302C  <->  [ok] - 剩余数量：238
4303  <->  [ok] - 剩余数量：237
4307  <->  [ok] - 剩余数量：236
4312C  <->  [ok] - 剩余数量：235
4321  <->  [ok] - 剩余数量：234
4326C  <->  [ok] - 剩余数量：233
4321C  <->  [ok] - 剩余数量：232
4329C  <->  [ok] - 剩余数量：231
4322  <->  [ok] - 剩余数量：230
4331C  <->  [ok] - 剩余数量：229
4340C  <->  [ok] - 剩余数量：228
4345C  <->  [ok] - 剩余数量：227
4361C  <->  [ok] - 剩余数量：226
4367C  <->  [ok] - 剩余数量：225
4365C  <->  [ok] - 剩余数量：224
4368  <->  [ok] - 剩余数量：223
4369C  <->  [ok] - 剩余数量：222
4375C  <->  [ok] - 剩余数量：221
4380C  <->  [ok] - 剩余数量：220
4390C  <->  [ok] - 剩余数量：219
4389C  <->  [ok] - 剩余数量：218
4392C  <->  [ok] - 剩余数量：217
4379C  <->  [ok] - 剩余数量：216
4393C  <->  [ok] - 剩余数量：21

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
